In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

## I - IMPORTATION DES DONNEES PATIENTS ET QUALITE DE L'AIR

In [2]:
#dataframe de tous les patients adultes en ile de france ainsi que leur pathologie
df_patients = pd.read_csv("data/data_cleaned/patients_FR_IDF_geocoded_adultes_clinique_patho.csv",sep=",",dtype={'codepost':str}).drop("Unnamed: 0",axis=1)


#dataframe des patients dont on a le statut de décès
df_deces = pd.read_csv("data/data_cleaned/date_dc_pseudo.csv", sep = ";")



In [3]:
len(df_deces)

65289

In [4]:
len(df_patients)

44770

### 1 - Jointure patients et  deces

In [ ]:
# year = [2017, 2018, 2019]
df_deces['date_derniere_nouvelle'] = pd.to_datetime(df_deces['date_derniere_nouvelle'])

#extraction des décès entre 2017 et 2019
pseudo = df_patients[df_patients['pseudo_provisoire'].isin(df_deces['pseudo_provisoire'])]['pseudo_provisoire']
df_deces = df_deces.drop_duplicates(subset='pseudo_provisoire', keep='last')
print(len(df_deces))

df_patients_w_statut= df_patients[df_patients['pseudo_provisoire'].isin(pseudo)].merge(df_deces, on='pseudo_provisoire', how='inner')
print(len(df_patients_w_statut))

df_patients_w_statut['date_derniere_nouvelle'] = df_patients_w_statut.apply(lambda x: pd.to_datetime("2019-12-31") if pd.isna(x['date_derniere_nouvelle']) else x['date_derniere_nouvelle'], axis=1)
patients_avant_2017 = df_patients_w_statut[df_patients_w_statut['date_derniere_nouvelle'].dt.year < 2017]
patients_apres_2019 = df_patients_w_statut[df_patients_w_statut['date_derniere_nouvelle'].dt.year > 2019]

#recuperer les patients dont la stae de derniere nouvelle est posterieure a 2016
df_patients_w_statut = df_patients_w_statut[df_patients_w_statut['date_derniere_nouvelle'].dt.year >= 2017]

64012
44764


In [6]:
df_patients_w_statut

,pseudo_provisoire,adresse,codepost,nom_commune_postal,requete,x,y,score,trust_score,street,...,ageaudiag,cancernum,date_diag,topo_initiale_cim10,topo_initialelib,patho,patient_co,date_naissance_y,date_derniere_nouvelle,statut_deces
0,1,34 RUE DES FRERES CHAUSSONS,92600.0,ASNIERES-SUR-SEINE,34 RUE DES FRERES CHAUSSONS 92600 ASNI...,2.289499,48.916298,0.832367,middle,34 Rue des Frères Chausson,...,32.0,1.0,2018-04-30,C92,Leucémie myéloïde aiguë,Hemato,10174,1985-06,2018-07-18,non
1,2,11 RUE EMILE DUBOIS,75014.0,PARIS,11 RUE EMILE DUBOIS 75014 PARIS,2.336628,48.831707,0.972567,high,11 Rue Emile Dubois,...,46.0,1.0,2017-07-17,C69,Tumeur maligne de la choroïde,Ophtalmo,10892,1970-10,2023-08-10,non
2,3,48 CHEMIN VERT,78680.0,EPONE,48 CHEMIN VERT 78680 EPONE,1.797376,48.950412,0.960861,high,48 Chemin Vert,...,69.0,1.0,2019-01-15,C61,Tumeur maligne de la prostate,Uro,8628,1949-03,2019-10-09,non
3,5,31 RUE DU GENERAL DE MIRIBEL,92500.0,RUEIL-MALMAISON,31 RUE DU GENERAL DE MIRIBEL 92500 RUEI...,2.173326,48.865232,0.973612,high,31 Rue du Général de Miribel,...,76.0,1.0,2018-02-22,C61,Tumeur maligne de la prostate,Uro,10174,1941-12,2024-01-15,non
4,6,124 RUE JEAN BAPTISTE CHARCOT,92400.0,COURBEVOIE,124 RUE JEAN BAPTISTE CHARCOT 92400 COUR...,2.273851,48.900542,0.978121,high,124 Rue Jean-Baptiste Charcot,...,50.0,1.0,2018-07-03,C50,Tumeur maligne à localisations contiguës du sein,Sein,10174,1968-03,2021-09-14,non
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44759,64286,15 RUE HOUSSA OUAID,77500,CHELLES,15 RUE HOUSSA OUAID 77500 CHELLES,2.571635,48.869009,0.803830,middle,15 Rue Houssa Ouïd,...,51.0,1.0,2004-06-28,C50,Tumeur maligne du mamelon et de l aréole,Sein,3018,1952-09,2023-05-12,oui
44760,64290,3 AV DE FOUILLEUSE,92210,SAINT-CLOUD,3 AV DE FOUILLEUSE 92210 SAIN...,2.211229,48.861208,0.749249,middle,3 Avenue de Fouilleuse,...,73.0,1.0,2015-06-15,C82,Autres formes de lymphome non hodgkinien folli...,Hemato,10174,1942-06,2024-04-17,non
44761,64292,60 RUE BAUDRICOURT,75013,Paris 13,60 RUE BAUDRICOURT 75013 Pari...,2.362960,48.825882,0.811235,middle,60 Rue Baudricourt,...,53.0,1.0,1998-06-09,C50,Tumeur maligne à localisations contiguës du sein,Sein,10892,1944-11,2024-03-20,non
44762,64293,159 AVENUE DE LA REPUBLIQUE,92320,CHATILLON,159 AVENUE DE LA REPUBLIQUE 92320 CHAT...,2.300784,48.809219,0.970667,high,159 Avenue de la République,...,54.0,1.0,2000-11-20,C50,Tumeur maligne du quadrant supéro-externe du sein,Sein,10174,1946-09,2024-05-23,non


In [9]:
df_patients_w_statut[df_patients_w_statut['date_derniere_nouvelle'].dt.year > 2019]

,pseudo_provisoire,adresse,codepost,nom_commune_postal,requete,x,y,score,trust_score,street,...,ageaudiag,cancernum,date_diag,topo_initiale_cim10,topo_initialelib,patho,patient_co,date_naissance_y,date_derniere_nouvelle,statut_deces
1,2,11 RUE EMILE DUBOIS,75014.0,PARIS,11 RUE EMILE DUBOIS 75014 PARIS,2.336628,48.831707,0.972567,high,11 Rue Emile Dubois,...,46.0,1.0,2017-07-17,C69,Tumeur maligne de la choroïde,Ophtalmo,10892,1970-10,2023-08-10,non
3,5,31 RUE DU GENERAL DE MIRIBEL,92500.0,RUEIL-MALMAISON,31 RUE DU GENERAL DE MIRIBEL 92500 RUEI...,2.173326,48.865232,0.973612,high,31 Rue du Général de Miribel,...,76.0,1.0,2018-02-22,C61,Tumeur maligne de la prostate,Uro,10174,1941-12,2024-01-15,non
4,6,124 RUE JEAN BAPTISTE CHARCOT,92400.0,COURBEVOIE,124 RUE JEAN BAPTISTE CHARCOT 92400 COUR...,2.273851,48.900542,0.978121,high,124 Rue Jean-Baptiste Charcot,...,50.0,1.0,2018-07-03,C50,Tumeur maligne à localisations contiguës du sein,Sein,10174,1968-03,2021-09-14,non
5,7,1 RUE RENAN,92600.0,ASNIERES-SUR-SEINE,1 RUE RENAN 92600 ASNI...,2.294234,48.915164,0.968157,high,1 Rue Renan,...,41.0,1.0,2018-08-09,C69,Tumeur maligne de la choroïde,Ophtalmo,10174,1976-10,2020-06-06,oui
6,12,6 RUE CARRIERE MARLE,92340.0,BOURG-LA-REINE,6 RUE CARRIERE MARLE 92340 BOUR...,2.320597,48.777321,0.962208,high,6 Rue Carrière Marlé,...,85.0,1.0,2019-01-02,C20,Tumeur maligne du rectum,Gastro,10174,1933-11,2020-03-04,oui
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44759,64286,15 RUE HOUSSA OUAID,77500,CHELLES,15 RUE HOUSSA OUAID 77500 CHELLES,2.571635,48.869009,0.803830,middle,15 Rue Houssa Ouïd,...,51.0,1.0,2004-06-28,C50,Tumeur maligne du mamelon et de l aréole,Sein,3018,1952-09,2023-05-12,oui
44760,64290,3 AV DE FOUILLEUSE,92210,SAINT-CLOUD,3 AV DE FOUILLEUSE 92210 SAIN...,2.211229,48.861208,0.749249,middle,3 Avenue de Fouilleuse,...,73.0,1.0,2015-06-15,C82,Autres formes de lymphome non hodgkinien folli...,Hemato,10174,1942-06,2024-04-17,non
44761,64292,60 RUE BAUDRICOURT,75013,Paris 13,60 RUE BAUDRICOURT 75013 Pari...,2.362960,48.825882,0.811235,middle,60 Rue Baudricourt,...,53.0,1.0,1998-06-09,C50,Tumeur maligne à localisations contiguës du sein,Sein,10892,1944-11,2024-03-20,non
44762,64293,159 AVENUE DE LA REPUBLIQUE,92320,CHATILLON,159 AVENUE DE LA REPUBLIQUE 92320 CHAT...,2.300784,48.809219,0.970667,high,159 Avenue de la République,...,54.0,1.0,2000-11-20,C50,Tumeur maligne du quadrant supéro-externe du sein,Sein,10174,1946-09,2024-05-23,non


In [11]:
df_patients_w_statut['statut_deces_a_letude'] = df_patients_w_statut.apply(
        lambda x: 'non' if (x['date_derniere_nouvelle'].year > 2019) & (x['statut_deces'] == 'oui') else x['statut_deces'], axis=1)

In [12]:
df_patients_w_statut[df_patients_w_statut['statut_deces_a_letude']=='oui']

,pseudo_provisoire,adresse,codepost,nom_commune_postal,requete,x,y,score,trust_score,street,...,cancernum,date_diag,topo_initiale_cim10,topo_initialelib,patho,patient_co,date_naissance_y,date_derniere_nouvelle,statut_deces,statut_deces_a_letude
39,60,18 RUE DE LYON,75012.0,PARIS,18 RUE DE LYON 75012 PARIS,2.371734,48.847805,0.979952,high,18 Rue de Lyon,...,1.0,2018-10-29,C50,Tumeur maligne du sein sans précision,Sein,10892,1932-06,2019-10-03,oui,oui
82,134,11 BIS RUE DE LA PLANCHE,75007.0,PARIS 7E ARRONDISSEMENT,11 BIS RUE DE LA PLANCHE 75007 PARI...,2.325159,48.852571,0.502327,middle,11b Rue de la Planche,...,1.0,2017-02-24,C96,Maladie de Letterer-Siwe,Hemato,10892,1978-11,2019-09-07,oui,oui
115,195,19 ALLE ANDR AMPERE,95200.0,SARCELLES,19 ALLE ANDR AMPERE 95200 SARCELLES,2.380362,48.976485,0.787768,middle,19 Allee Andre Ampere,...,1.0,2013-10-15,C70,Tumeur maligne des méninges sans précision,Neuro,2824,1966-07,2019-01-08,oui,oui
117,197,7 RUE ROUSSELET,75007.0,Paris 07,7 RUE ROUSSELET 75007 Pari...,2.318040,48.849719,0.794246,middle,7 Rue Rousselet,...,1.0,2018-05-02,C34,Tumeur maligne du lobe supérieur bronches ou p...,Thorax,10892,1943-04,2019-04-24,oui,oui
120,200,19 RUE HENRI REGNAULT,92380.0,GARCHES,19 RUE HENRI REGNAULT 92380 GARCHES,2.189318,48.847439,0.965825,high,19 Rue Henri Regnault,...,1.0,2017-02-28,C10,Tumeur maligne à localisations contiguës de l ...,ORL,10174,1949-03,2017-12-30,oui,oui
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44239,63509,13 RUE PAUL RENAUD,93140,BONDY,13 RUE PAUL RENAUD 93140 BONDY,2.479027,48.910257,0.971355,high,13 Rue Paul Renaud,...,1.0,2001-11-28,C61,Tumeur maligne de la prostate,Uro,3204,1928-12,2018-03-26,oui,oui
44248,63520,CHEMIN DE LA GUEULE A VACHE,95270,ASNIERES SUR OISE,CHEMIN DE LA GUEULE A VACHE 95270 ASNI...,2.351422,49.136059,0.812973,middle,Chemin de la Gueule a Vaches,...,1.0,2001-09-15,C61,Tumeur maligne de la prostate,Uro,2824,1932-08,2019-10-16,oui,oui
44258,63536,82 AVENUE SIMON BOLEVAR,75019,PARIS,82 AVENUE SIMON BOLEVAR 75019 PARIS,2.374718,48.876771,0.799921,middle,82 Avenue Simon Bolivar,...,1.0,2009-02-17,C50,Tumeur maligne du mamelon et de l aréole,Sein,10892,1926-10,2018-10-03,oui,oui
44567,64024,3 RUE LEON BLUM,92260,FONTENAY AUX ROSES,3 RUE LEON BLUM 92260 FONT...,2.293751,48.791088,0.964969,high,3 Rue Léon Blum,...,1.0,2010-06-01,C50,Tumeur maligne du quadrant supéro-externe du sein,Sein,10174,1936-07,2019-06-15,oui,oui


In [13]:
df_patients_w_statut[df_patients_w_statut['statut_deces_a_letude']=='non']

,pseudo_provisoire,adresse,codepost,nom_commune_postal,requete,x,y,score,trust_score,street,...,cancernum,date_diag,topo_initiale_cim10,topo_initialelib,patho,patient_co,date_naissance_y,date_derniere_nouvelle,statut_deces,statut_deces_a_letude
0,1,34 RUE DES FRERES CHAUSSONS,92600.0,ASNIERES-SUR-SEINE,34 RUE DES FRERES CHAUSSONS 92600 ASNI...,2.289499,48.916298,0.832367,middle,34 Rue des Frères Chausson,...,1.0,2018-04-30,C92,Leucémie myéloïde aiguë,Hemato,10174,1985-06,2018-07-18,non,non
1,2,11 RUE EMILE DUBOIS,75014.0,PARIS,11 RUE EMILE DUBOIS 75014 PARIS,2.336628,48.831707,0.972567,high,11 Rue Emile Dubois,...,1.0,2017-07-17,C69,Tumeur maligne de la choroïde,Ophtalmo,10892,1970-10,2023-08-10,non,non
2,3,48 CHEMIN VERT,78680.0,EPONE,48 CHEMIN VERT 78680 EPONE,1.797376,48.950412,0.960861,high,48 Chemin Vert,...,1.0,2019-01-15,C61,Tumeur maligne de la prostate,Uro,8628,1949-03,2019-10-09,non,non
3,5,31 RUE DU GENERAL DE MIRIBEL,92500.0,RUEIL-MALMAISON,31 RUE DU GENERAL DE MIRIBEL 92500 RUEI...,2.173326,48.865232,0.973612,high,31 Rue du Général de Miribel,...,1.0,2018-02-22,C61,Tumeur maligne de la prostate,Uro,10174,1941-12,2024-01-15,non,non
4,6,124 RUE JEAN BAPTISTE CHARCOT,92400.0,COURBEVOIE,124 RUE JEAN BAPTISTE CHARCOT 92400 COUR...,2.273851,48.900542,0.978121,high,124 Rue Jean-Baptiste Charcot,...,1.0,2018-07-03,C50,Tumeur maligne à localisations contiguës du sein,Sein,10174,1968-03,2021-09-14,non,non
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44759,64286,15 RUE HOUSSA OUAID,77500,CHELLES,15 RUE HOUSSA OUAID 77500 CHELLES,2.571635,48.869009,0.803830,middle,15 Rue Houssa Ouïd,...,1.0,2004-06-28,C50,Tumeur maligne du mamelon et de l aréole,Sein,3018,1952-09,2023-05-12,oui,non
44760,64290,3 AV DE FOUILLEUSE,92210,SAINT-CLOUD,3 AV DE FOUILLEUSE 92210 SAIN...,2.211229,48.861208,0.749249,middle,3 Avenue de Fouilleuse,...,1.0,2015-06-15,C82,Autres formes de lymphome non hodgkinien folli...,Hemato,10174,1942-06,2024-04-17,non,non
44761,64292,60 RUE BAUDRICOURT,75013,Paris 13,60 RUE BAUDRICOURT 75013 Pari...,2.362960,48.825882,0.811235,middle,60 Rue Baudricourt,...,1.0,1998-06-09,C50,Tumeur maligne à localisations contiguës du sein,Sein,10892,1944-11,2024-03-20,non,non
44762,64293,159 AVENUE DE LA REPUBLIQUE,92320,CHATILLON,159 AVENUE DE LA REPUBLIQUE 92320 CHAT...,2.300784,48.809219,0.970667,high,159 Avenue de la République,...,1.0,2000-11-20,C50,Tumeur maligne du quadrant supéro-externe du sein,Sein,10174,1946-09,2024-05-23,non,non


### 2 - Resumé statistiques des données patients

In [ ]:
print("RESUME STATISTIQUE DES DONNEES PATIENTS: \n")

print(f"1 - Nous avons au total : {len(df_patients)} patients adultes en ile de france")
print(f"2 - Dont {len(df_patients_w_statut)} ont un statut de décès connu c'est à dire nous savons s'ils sont décédés ou non")
print(f"3 - Parmi ces patients, {len(df_patients_w_statut[df_patients_w_statut].dt.year >= 2017)} ont une date de derniere nouvelle postérieure au 1er janvier 2017 et donc sont pris en compte dans l'étude")
print(f"4 - Sur cette période, {len(df_patients_w_statut[df_patients_w_statut['statut_deces_a_letude']=='oui'])} sont décédés et {len(df_patients_w_statut[df_patients_w_statut['statut_deces_a_letude']=='non'])} ne le sont pas")

RESUME STATISTIQUE DES DONNEES PATIENTS: 

1 - Nous avons au total : 44770 patients adultes en ile de france
2 - Dont 44268 ont un statut de décès connu c'est à dire nous savons s'ils sont décédés ou non
3 - Parmi ces patients, 44268 ont une date de derniere nouvelle apres 2017
4 - Sur cette période, 3990 sont décédés et 40278 ne le sont pas


In [17]:
len(df_patients_w_statut)

44268

In [ ]:
df_patients_w_statut.groupby('patient_sexe')['pseudo_provisoire'].count().reset_index(name='nombre_deces')

,patient_sexe,nombre_deces
0,F,35414
1,M,8854


: 

In [18]:
df_patients_w_statut.to_csv("data/data_cleaned/patients_FR_IDF_geocoded_adultes_clinique_patho_statut_deces_apres_2017.csv", sep=",", index=False)

In [ ]:
# Concentration moyenne mensuelle par polluant et par patient pour exposition aigu et chronique 
# On a 4 fichiers, un par polluant, contenant la date et le pseudo_provisoire, mois, année, la concentration moyenne mensuelle pollution aigue et chronique 

pm10 = pd.read_csv("data/data_air/PM10_airparif_moyenne_mensuelle.csv", sep=";")
o3 = pd.read_csv("data/data_air/O3_airparif_moyenne_mensuelle.csv", sep=";")
no2 = pd.read_csv("data/data_air/NO2_airparif_moyenne_mensuelle.csv", sep=";")
pm25 = pd.read_csv("data/data_air/PM25_airparif_moyenne_mensuelle.csv", sep=";")


# df_patients_peaks = pd.read_csv("data/data_air/patients_nbre_peaks.csv",sep=";")




In [ ]:
# Nombre de peaks mensuels par polluant et par patient pour exposition aigu (pseudo_provisoire, mois, année, nombre de peaks pm10, nombre de peaks pm2.5, nombre de peaks o3, nombre de peaks no2)
df_patients_peaks_mensuels = pd.read_csv("C:/patients_peaks_mensuels.csv",sep=";")

#infos concernant la pollution chronique 
df_pollution_chron = pd.read_csv("C:/exposition_chronique.csv", usecols=['pseudo_provisoire', 'annee', 'exposition_chronique_pm10', 'exposition_chronique_pm25', 'jours_depassement_no2', 'exposition_chronique_no2', 'jours_depassement_o3', 'exposition_chronique_o3'], sep=';', dtype={'codepost': str})

In [ ]:
# Fusionne progressivement les données de qualité de l'air
print(len(df_patients_w_statut))
df = df_patients_w_statut.merge(pm10[["pseudo_provisoire", "annee", "mois","moyenne_PM10", "moyenne_PM10_chron"]], on=["pseudo_provisoire"], how="left")
print(len(df)/36)
# df = df.merge(patients_peaks_polluants[["pseudo_provisoire","DATE","peaks_pm10","peaks_pm25","peaks_o3","peaks_no2"]], on=["pseudo_provisoire","DATE"], how="left")
df = df.merge(df_patients_peaks_mensuels[["pseudo_provisoire","annee","mois","nb_peaks_no2", "nb_peaks_o3", "nb_peaks_pm10", "nb_peaks_pm25"]], on=["pseudo_provisoire","annee","mois"], how="left")
print(len(df)/36)
df = df.merge(o3[["pseudo_provisoire", "annee", "mois","moyenne_O3", "moyenne_O3_chron"]], on=["pseudo_provisoire", "annee", "mois"], how="left")
print(len(df)/36)
df = df.merge(no2[["pseudo_provisoire", "annee", "mois","moyenne_NO2", "moyenne_NO2_chron"]], on=["pseudo_provisoire", "annee", "mois"], how="left")
print(len(df)/36)
df = df.merge(pm25[["pseudo_provisoire", "annee", "mois","moyenne_PM25", "moyenne_PM25_chron"]], on=["pseudo_provisoire", "annee", "mois"], how="left")
print(len(df)/36)
df = df.merge(df_pollution_chron, on=["pseudo_provisoire", "annee"], how="left")
print(len(df)/36)



44268
44268.0
44268.0
44268.0
44268.0
44268.0
44268.0


In [38]:
len(df)

1593648

In [39]:
df[df.isnull().any(axis=1)].drop_duplicates(subset='pseudo_provisoire', keep='last')

,pseudo_provisoire,adresse,codepost,nom_commune_postal,requete,x,y,score,trust_score,street,...,moyenne_NO2,moyenne_NO2_chron,moyenne_PM25,moyenne_PM25_chron,exposition_chronique_pm10,exposition_chronique_pm25,jours_depassement_no2,exposition_chronique_no2,jours_depassement_o3,exposition_chronique_o3
16127,692,9 RUE PAUL BERT,92320.0,CHATILLON,9 RUE PAUL BERT 92320 CHAT...,2.290437,48.803982,0.962734,high,9 Rue Paul Bert,...,43.332661,29.153226,12.040323,11.985887,0,0,194,1,26,1
34991,1488,115 AVENUE JOFFRE,93800.0,EPINAY-SUR-SEINE,115 AVENUE JOFFRE 93800 EPIN...,2.295175,48.961509,0.975788,high,115 Avenue Joffre,...,51.282258,33.719758,12.481855,13.282258,0,0,221,1,26,1
41471,1750,BUREAU MILITAIRE/ AMBASSSADE KOWEIT75 AVENUE D...,75008.0,Paris 08,BUREAU MILITAIRE/ AMBASSSADE KOWEIT75 AVENUE D...,2.302859,48.871285,0.451735,middle,Avenue des Champs Elysées,...,56.401210,38.129032,13.453629,14.582661,0,0,265,1,19,0
61955,2552,35 GRANDE RUE,91590.0,BOISSY LE CUTTE,35 GRANDE RUE 91590 BOIS...,2.282848,48.471173,0.954024,high,35 Grande Rue,...,20.943548,14.637097,8.971774,8.133065,0,0,32,1,33,1
138131,5385,44 RUE PAUL CURIEN,78130.0,LES MUREAUX,44 RUE PAUL CURIEN 78130 LES ...,1.900378,48.993375,0.972258,high,44 Rue Paul Curien,...,41.506048,20.266129,10.991935,10.268145,0,0,131,1,28,1
141227,5502,15 BD PAUL EMILE VICTOR,92200.0,NEUILLY-SUR-SEINE,15 BD PAUL EMILE VICTOR 92200 NEUI...,2.264748,48.894642,0.740626,middle,15 boulevard paul-émile victor,...,55.806452,35.078629,13.542339,13.274194,0,0,253,1,20,0
166499,6442,RUE E 62 BP 1185 ...,99326.0,ABIDJAN 06,RUE E 62 BP 1185 ...,2.328659,48.851506,0.177850,low,Voie E/6,...,54.858871,40.699597,13.207661,14.709677,0,0,259,1,20,0
177587,6861,175 BOULEVARD MURAT,75016.0,PARIS,175 BOULEVARD MURAT 75016 PARIS,2.264152,48.838924,0.981714,high,175 Boulevard Murat,...,48.641129,32.600806,12.995968,12.064516,0,0,230,1,20,0
210959,8140,91 AVENUE JOLIOT CURIE,92000.0,NANTERRE,91 AVENUE JOLIOT CURIE 92000 NANT...,2.210316,48.893428,0.609010,middle,91 Avenue Frédéric et Irène Joliot Curie,...,51.479839,30.032258,12.917339,12.679435,0,0,226,1,20,0
213551,8251,71 ROUTE DE MANTES,78200.0,BUCHELAY,71 ROUTE DE MANTES 78200 BUCH...,1.682205,48.978786,0.960127,high,71 Route de Mantes,...,35.028226,18.967742,9.957661,8.760081,0,0,91,1,25,0


In [41]:
df[df['statut_deces_a_letude']== 'oui'].drop_duplicates(subset='pseudo_provisoire', keep='last')

,pseudo_provisoire,adresse,codepost,nom_commune_postal,requete,x,y,score,trust_score,street,...,moyenne_NO2,moyenne_NO2_chron,moyenne_PM25,moyenne_PM25_chron,exposition_chronique_pm10,exposition_chronique_pm25,jours_depassement_no2,exposition_chronique_no2,jours_depassement_o3,exposition_chronique_o3
1439,60,18 RUE DE LYON,75012.0,PARIS,18 RUE DE LYON 75012 PARIS,2.371734,48.847805,0.979952,high,18 Rue de Lyon,...,56.296371,39.788306,13.477823,14.772177,0,0,269,1,21,0
2987,134,11 BIS RUE DE LA PLANCHE,75007.0,PARIS 7E ARRONDISSEMENT,11 BIS RUE DE LA PLANCHE 75007 PARI...,2.325159,48.852571,0.502327,middle,11b Rue de la Planche,...,54.901210,31.241935,13.199597,12.826613,0,0,258,1,20,0
4175,195,19 ALLE ANDR AMPERE,95200.0,SARCELLES,19 ALLE ANDR AMPERE 95200 SARCELLES,2.380362,48.976485,0.787768,middle,19 Allee Andre Ampere,...,51.862903,30.838710,12.054435,12.883065,0,0,213,1,31,1
4247,197,7 RUE ROUSSELET,75007.0,Paris 07,7 RUE ROUSSELET 75007 Pari...,2.318040,48.849719,0.794246,middle,7 Rue Rousselet,...,54.048387,31.487903,13.129032,12.752016,0,0,251,1,21,0
4355,200,19 RUE HENRI REGNAULT,92380.0,GARCHES,19 RUE HENRI REGNAULT 92380 GARCHES,2.189318,48.847439,0.965825,high,19 Rue Henri Regnault,...,43.088710,20.326613,11.461694,10.413306,0,0,185,1,27,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1574819,63509,13 RUE PAUL RENAUD,93140,BONDY,13 RUE PAUL RENAUD 93140 BONDY,2.479027,48.910257,0.971355,high,13 Rue Paul Renaud,...,55.465726,38.647177,12.850806,13.296371,0,0,257,1,27,1
1575143,63520,CHEMIN DE LA GUEULE A VACHE,95270,ASNIERES SUR OISE,CHEMIN DE LA GUEULE A VACHE 95270 ASNI...,2.351422,49.136059,0.812973,middle,Chemin de la Gueule a Vaches,...,36.348790,18.981855,9.935484,9.921371,0,0,106,1,34,1
1575503,63536,82 AVENUE SIMON BOLEVAR,75019,PARIS,82 AVENUE SIMON BOLEVAR 75019 PARIS,2.374718,48.876771,0.799921,middle,82 Avenue Simon Bolivar,...,57.719758,42.637097,13.814516,15.187500,0,0,278,1,21,0
1586591,64024,3 RUE LEON BLUM,92260,FONTENAY AUX ROSES,3 RUE LEON BLUM 92260 FONT...,2.293751,48.791088,0.964969,high,3 Rue Léon Blum,...,42.399194,25.252016,11.921371,11.590726,0,0,188,1,31,1


In [43]:
df.to_csv("data/data_cleaned/patients_FR_IDF_geocoded_adultes_clinique_patho_pollution_mensuelles.csv", sep=";", index=False)